In [1]:
#table 7 IE descriptive

import pandas as pd
import numpy as np
import matplotlib as plt
#!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels as sm

#2 econtools
#!pip install econtools
import econtools as econ
import econtools.metrics as mt

In [2]:
df = pd.read_stata('data/IE.dta')
print(df.shape)

df = df.sort_values(by='authority_code',ascending=True)

(2537, 30)


In [3]:
#preserve
'''df['lmiles_pa_to'] = np.log(df['miles_pa_torino']+1)
del df['experience']

df = df.sort_values(by = 'authority_code', ascending = True)
exp = df['authority_code'].value_counts()
exp = pd.DataFrame({'auth':exp.index, 'value':exp.values})

for i in range(len(df)):
    for j in range(len(exp)):
        if df.loc[i,'authority_code'] == exp.loc[j, 'auth']:
            df.loc[i, 'experience'] = exp.loc[j, 'value']

df['reserve_price'] = df['reserve_price']/1000
df['population'] = df['population']/1000
df= df[['fpsb_auction', 'forcedfp_strict', 'reserve_price', 'contract_duration', 'experience', 'population', 'discount', 'n_bidders', 'miles_pa_torino', 'miles_firm_work', 'days_to_award']]
idx = df[df['forcedfp_strict']!=1].index
df = df.drop(idx)

'''

"df['lmiles_pa_to'] = np.log(df['miles_pa_torino']+1)\ndel df['experience']\n\ndf = df.sort_values(by = 'authority_code', ascending = True)\nexp = df['authority_code'].value_counts()\nexp = pd.DataFrame({'auth':exp.index, 'value':exp.values})\n\nfor i in range(len(df)):\n    for j in range(len(exp)):\n        if df.loc[i,'authority_code'] == exp.loc[j, 'auth']:\n            df.loc[i, 'experience'] = exp.loc[j, 'value']\n\ndf['reserve_price'] = df['reserve_price']/1000\ndf['population'] = df['population']/1000\ndf= df[['fpsb_auction', 'forcedfp_strict', 'reserve_price', 'contract_duration', 'experience', 'population', 'discount', 'n_bidders', 'miles_pa_torino', 'miles_firm_work', 'days_to_award']]\nidx = df[df['forcedfp_strict']!=1].index\ndf = df.drop(idx)\n\n"

In [4]:
df = df[(df['sample']=="Telemat_MyCS_2009") | (df['sample']=="Telemat_2011") |  (df['sample']=="Telemat_2010")]
df['lmiles_pa_to'] = np.log(df['miles_pa_torino']+1)
del df['experience']

df = df.sort_values(by = 'authority_code', ascending = True)
exp = df['authority_code'].value_counts()
exp = pd.DataFrame({'auth':exp.index, 'value':exp.values})
for i in range(len(df)):
    for j in range(len(exp)):
        if df.loc[i,'authority_code'] == exp.loc[j, 'auth']:
            df.loc[i, 'experience'] = exp.loc[j, 'value']

In [5]:
auth_type_list = pd.get_dummies(df['auth_type_region'])
df = pd.concat([df, auth_type_list],axis = 1)

In [6]:
reg_col = ['lmiles_pa_to', 'population', 'experience', 'municipality','logreserve_price', 'contract_duration']
for i in auth_type_list:
    reg_col.append(i)
#emilia lomagna should be omitted

In [62]:
sample = 'forcedfp_strict'
outcome = ['discount', 'days_to_award']

#for o in outcome:
df_probit = df[df[sample]==0]
probit_mod = sm.discrete.discrete_model.Probit(df_probit['fpsb_auction'], df_probit[reg_col],missing = 'drop')
probit_res = probit_mod.fit()
print(probit_res.summary())
#probit value checked
df_probit['proscore'] = probit_res.predict(df_probit['fpsb_auction']) #error
df_probit.loc[(df_probit['proscore']>0.925)|(df_probit['proscore']<0.075), 'proscore'] = np.nan

Optimization terminated successfully.
         Current function value: 0.244893
         Iterations 8
                          Probit Regression Results                           
Dep. Variable:           fpsb_auction   No. Observations:                 1604
Model:                         Probit   Df Residuals:                     1593
Method:                           MLE   Df Model:                           10
Date:                Wed, 08 Jul 2020   Pseudo R-squ.:                  0.5645
Time:                        15:07:42   Log-Likelihood:                -392.81
converged:                       True   LL-Null:                       -901.99
Covariance Type:            nonrobust   LLR p-value:                2.078e-212
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
lmiles_pa_to         -0.0229      0.096     -0.238      0.812      -0.212       0.166
populati

ValueError: shapes (1,2142) and (11,) not aligned: 2142 (dim 1) != 11 (dim 0)

In [63]:
work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)
df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

#probit reg_col에 이어서 붙이면 됨
reg_col = reg_col.append('fpsb_acution')
for i in work_list:
    reg_col.append(i)
for j in year_list:
    reg_col.append(j)

In [64]:
if df[sample]==0 and df['proscore'].isnull()==False:
    for o in outcome:
        fe_reg = mt.reg(df[o],df[reg_col], fe_name = 'authority_code', cluster = 'auth_anno', check_colinear = True)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [47]:
df_probit.head()

,authority_code,auth_anno,fpsb_auction,n_bidders,discount,days_to_award,work_category,reserve_price,logreserve_price,above1,...,color_ruling_party,ruling_party_advantage,golenpicci05index,lmiles_pa_to,experience,EMILIA ROMAGNA,LIGURIA,LOMBARDIA,PIEMONTE,VENETO
630,2.0,8.0,0.0,26.0,14.510000,NaN,OG03,145495.0,11.887897,0.0,...,Right,38.32,0.552,3.900962,17.0,0,0,0,1,0
95,2.0,10.0,0.0,91.0,16.044001,NaN,OG03,236900.0,12.375393,0.0,...,Right,38.32,0.552,3.900962,17.0,0,0,0,1,0
1954,2.0,11.0,0.0,23.0,22.799999,NaN,OG03,623500.0,13.343104,0.0,...,Right,38.32,0.552,3.900962,17.0,0,0,0,1,0
1017,2.0,8.0,0.0,12.0,8.782000,4.0,OG03,87580.0,11.380308,0.0,...,Right,38.32,0.552,3.900962,17.0,0,0,0,1,0
2197,2.0,12.0,0.0,60.0,19.350000,NaN,OG03,243000.0,12.400817,0.0,...,Right,38.32,0.552,3.900962,17.0,0,0,0,1,0
